<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex04notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex04notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)

※ この notebook は，授業時間中の解説や板書と併用することを想定して作っていますので，説明が不十分なところが多々あります．


----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso

---
### California Housing Dataset

reportA でも使った California Housing Dataset を使う．

In [ ]:
# データを入手
data = fetch_california_housing(as_frame=True)
dfX = data['data']
dfY = data['target']

# 全データの 8 割を学習データとし，残りを検証データとする
#   random_state を指定しなければ実行のたびに分割の仕方が変わるので毎回異なる実験結果となるが，
#   ここでは一つの値に固定しているで，何度やっても（当然，誰がやっても）同じ条件で実験できる
XL, XV, yL, yV = train_test_split(dfX.to_numpy(), dfY.to_numpy(), train_size=0.8, random_state=0)
print(XL.shape, yL.shape, XV.shape, yV.shape)

In [ ]:
# 標準化
scaler = StandardScaler()
scaler.fit(XL)
print(scaler.mean_) # 平均
print(scaler.var_) # 分散

# 求めた平均と分散を使って XL, XV を標準化
XL2 = scaler.transform(XL)
XV2 = scaler.transform(XV)

---
## ラッソ回帰
---

---
### 特徴を選択したい

線形回帰モデルを学習させると，次のようになる．

In [ ]:
# 線形回帰モデルの学習
model = LinearRegression()
regressor = model.fit(XL2, yL)

# パラメータ表示用
dfW = pd.DataFrame(columns=['alpha', 'const']+list(dfX.columns))
dfW['alpha'] = [0.0]
dfW['const'] = regressor.intercept_
dfW.iloc[0, 2:] = regressor.coef_
#dfW.set_index('alpha', inplace=True)

# 平均二乗誤差の計算
yL_pred = regressor.predict(XL2)
msqeL = np.mean((yL_pred - yL)**2)
yV_pred = regressor.predict(XV2)
msqeV = np.mean((yV_pred - yV)**2)

# 出力
print(f'msqeL = {msqeL:.4f}  msqeV = {msqeV:.4f}')
dfW

この場合，得られたモデルはおよそ

$$
\mbox{（住宅価格の予測値）} = 2.07 + 0.82\times (\mbox{MedInc}) + 0.12\times (\mbox{HouseAge}) + \cdots +(-0.87)\times \mbox{(Longtitude)}  
$$

という式である．'MedInc' から 'Longtitude' までの8つの変数に対する係数はどれも $0$ ではないので，住宅価格の予測に寄与しているということになる．
しかし，これらの変数の中には，住宅価格に大きな影響を与える重要なものもあれば，影響の少ない重要でないものもあるかもしれない．

このような問題意識を一般化すると，「与えられた多変量データの中から重要な変数/特徴を見つけ出したい」ということになる．このようなことを行うのが **特徴選択** (**feature selection**) である．

特徴選択の方法には様々なものがあるが，ここではその中のひとつであるラッソ回帰を取り上げる．

---
### ラッソ回帰とは

**ラッソ回帰** （**LASSO回帰**, LASSO は "least absolute shrinkage and selection operator"の頭文字）は，汎化性能の改善と特徴選択の実現という二つを主な目的とした回帰の手法である．

$\mathbf{x}$ から $y$ を予測する回帰モデル $f(\mathbf{x})$ を考える．
このモデルの学習のためのデータを $ (\mathbf{x}_n, y_n)$ ($n = 1, 2, \ldots, N$) とする．

最小二乗法による線形回帰のように，モデルの出力ととその正解の値との間の二乗誤差の和のみを最小化する場合，
学習の目的関数は次式のように書けた．

$$
E(\mathbf{w}) =
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2
$$

リッジ回帰の場合，モデルパラメータの値の大きさに制約を加えるために，目的関数に次のような正則化項を付加するのだった．

$$
E(\mathbf{w}) =
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 + \alpha ||\mathbf{w}||^2
$$

ここで，$\mathbf{w}$ はモデルパラメータを並べたベクトルである．パラメータ数を $M$ として，$\mathbf{w} = (w_1, w_2, \ldots, w_M)$ とおくと，上の式は次のように書き直せる．

$$
E(\mathbf{w}) =
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 + \alpha \sum_{m=1}^{M}w_m^2
$$

正則化項は，パラメータの二乗和を小さくする働きをしている．


これに対して，ラッソ回帰では，パラメータの絶対値の和を小さくする．
すなわち，次のような目的関数を最小化する．

$$
E(\mathbf{w}) =
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 + \alpha \sum_{m=1}^{M}|w_m|
$$


---
### やってみよう

California Housing Dataset を使ってラッソ回帰の実験をやってみよう．
ここでは，[sklearn.linear_model.Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) を用いる．

In [ ]:
# 正則化項の係数
alpha = 0.01  ### ここを少しずつ大きくしていってみよう

# LASSO回帰モデルの学習
model = Lasso(alpha=alpha)
regressor = model.fit(XL2, yL)

# パラメータ表示用
dfW = pd.DataFrame(columns=['alpha', 'const']+list(dfX.columns))
dfW['alpha'] = [0.0]
dfW['const'] = regressor.intercept_
dfW.iloc[0, 2:] = regressor.coef_
dfW.set_index('alpha', inplace=True)

# 平均二乗誤差の計算
yL_pred = regressor.predict(XL2)
msqeL = np.mean((yL_pred - yL)**2)
yV_pred = regressor.predict(XV2)
msqeV = np.mean((yV_pred - yV)**2)

# 出力
print(f'msqeL = {msqeL:.4f}  msqeV = {msqeV:.4f}')
dfW

正則化項の係数 `alpha` を大きくしていくと，パラメータの値や平均二乗誤差の値はどのように変化するだろうか．

次のセルを実行すると，いろいろな `alpha` の設定での実験をまとめて行うことができる．

In [ ]:
alphaList = np.logspace(-3, 0, num=10)
N, D = XL2.shape
w = np.empty((len(alphaList), D))

dfW = pd.DataFrame(columns=['alpha', 'msqeL', 'msqeV', 'const']+list(dfX.columns))
dfW['alpha'] = alphaList

for i, alpha in enumerate(alphaList):
    model = Lasso(alpha=alpha)
    regressor = model.fit(XL2, yL)
    dfW.iloc[i, 1] = np.mean((regressor.predict(XL2) - yL)**2)
    dfW.iloc[i, 2] = np.mean((regressor.predict(XV2) - yV)**2)
    dfW.iloc[i, 3] = regressor.intercept_
    dfW.iloc[i, 4:] = regressor.coef_

fig, ax = plt.subplots(1, 1, figsize=(8, 4))
w = dfW.iloc[:, 3:].to_numpy()
for d in range(D):
    ax.plot(alphaList, w[:, d], marker='o')
ax.set_xscale('log')
ax.set_ylim(-1.0, 1.0)
ax.set_xlabel(r'$\alpha$')
plt.show()

dfW.set_index('alpha', inplace=True)
dfW

住宅価格の予測のために重要な特徴を（定数項を除いて）4つあげるとすると，どれだろう．

---
### いろいろ



#### スパース

**スパース** (sparse)


#### $p$-ノルム

$ \mathbf{x} = (x_1, x_2, \ldots, x_M)$ のとき，

$$
||\mathbf{x}||_p = \left( \sum_{m=1}^{M} |x_d|^p \right)^{\frac{1}{p}}
$$

を $L^{p}$ ノルムまたは $p$-ノルムという．$p=2$ のときはユークリッドノルム．

リッジ回帰はパラメータの $L^2$ ノルム（ユークリッドノルム）を,
ラッソ回帰はパラメータの $L^1$ ノルムを制約する．

$$
\begin{aligned}
\textrm{Ridge:}  E(\mathbf{w}) &=
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 + \alpha ||\mathbf{w}||_2^2\\
\textrm{LASSO:} E(\mathbf{w}) & =\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 + \alpha ||\mathbf{w}||_1\\
\end{aligned}
$$


#### なぜ $L^1$ ノルム最小化でスパースになるのか